In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
def calCorrelation(a, b, windowSize, stepSize):
    def calTempCorrelation(a, b):
        numerator = 0
        dedominator = 0
        tempA = 0
        tempB = 0
        for i, j in zip(a, b):
            numerator += np.abs(i)*j
            tempA += i*i
            tempB += j*j
        dedominator = np.sqrt(tempA*tempB)
        return numerator/dedominator
    corr = []
    for i in range(0, len(a), stepSize):
        corr.append(calTempCorrelation(a[i: i+windowSize], b[i: i+windowSize]))
    return corr

In [3]:
def activitySplit(csvName, windowSize, stepSize, corrThresholdLength):
    joinWindowSize = windowSize  # 两个窗口拼接成一个时间的阈值
    folderPath = r"E:\Jupyter\实验室\采集数据\yqcc"
    csvPath = folderPath + "\\" + csvName
    dataXYZ = pd.read_csv(csvPath, header= 0)
    corrXYs = calCorrelation(dataXYZ.iloc[:, 0], dataXYZ.iloc[:, 1], windowSize, stepSize) 
    activityPosList = []
    isFirst = True
    
    corrThresholdList = corrXYs[: corrThresholdLength]
    corrThresholdListCopy = corrThresholdList.copy()
    corrThresholdListCopy.sort()
    percent25 = corrThresholdListCopy[int(0.25*corrThresholdLength)]
    percent75 = corrThresholdListCopy[int(0.75*corrThresholdLength)]
    
    coef = 3
    corrThreshold = np.abs(percent75 + coef*(np.abs(percent75 - percent25)))
    
    for i, corrXY in enumerate(corrXYs):
        if np.abs(corrXY) <= corrThreshold:
            if isFirst:
                start = i * stepSize
                end = start + windowSize
                isFirst = False
            else:
                if i*stepSize - end <= joinWindowSize:
                    end = i*stepSize + windowSize
                else:
                    activityPosList.append(str(start) + " " + str(end))
                    start = i * stepSize
                    end = start + windowSize
        corrThresholdList.pop(0)
        corrThresholdList.append(corrXY)
        corrThresholdListCopy = corrThresholdList.copy()
        corrThresholdListCopy.sort()
        percent25 = corrThresholdListCopy[int(0.25*corrThresholdLength)]
        percent75 = corrThresholdListCopy[int(0.75*corrThresholdLength)]
        corrThreshold = np.abs(percent75 + coef*(np.abs(percent75 - percent25)))

    print(csvName, len(activityPosList))
    return len(activityPosList)
    


In [4]:
%%time
csvPath = r"E:\Jupyter\实验室\采集数据\yqcc"
csvLists = os.listdir(csvPath)

for csvName in csvLists:
    activitySplit(csvName, 32, 16, 500)
    

dig_1.0_A.csv 54
dig_1.0_B.csv 50
dig_1.0_C.csv 52
dig_1.0_D.csv 47
dig_1.0_E.csv 52
dig_1.0_F.csv 43
dig_2.0_A.csv 47
dig_2.0_B.csv 35
dig_2.0_C.csv 44
dig_2.0_D.csv 49
dig_2.0_E.csv 46
dig_2.0_F.csv 47
dig_3.0_A.csv 49
dig_3.0_B.csv 51
dig_3.0_C.csv 37
dig_3.0_D.csv 47
dig_3.0_E.csv 39
dig_3.0_F.csv 38
dig_4.0_A.csv 42
dig_4.0_B.csv 56
dig_4.0_C.csv 23
dig_4.0_D.csv 47
dig_4.0_E.csv 26
dig_4.0_F.csv 36
dig_5.0_A.csv 40
dig_5.0_B.csv 31
dig_5.0_C.csv 20
dig_5.0_D.csv 22
dig_5.0_E.csv 22
dig_5.0_F.csv 26
dig_6.0_A.csv 32
dig_6.0_B.csv 24
dig_6.0_C.csv 27
dig_6.0_D.csv 39
dig_6.0_E.csv 17
dig_6.0_F.csv 22
dig_7.0_A.csv 35
dig_7.0_B.csv 22
dig_7.0_C.csv 10
dig_7.0_D.csv 37
dig_7.0_E.csv 11
dig_7.0_F.csv 14
dig_8.0_A.csv 36
dig_8.0_B.csv 26
dig_8.0_C.csv 13
dig_8.0_D.csv 45
dig_8.0_E.csv 11
dig_8.0_F.csv 13
jumpVarity_A.csv 23
jump_1.0_A.csv 57
jump_1.0_B.csv 70
jump_1.0_C.csv 50
jump_1.0_D.csv 46
jump_1.0_E.csv 59
jump_1.0_F.csv 51
jump_2.0_A.csv 56
jump_2.0_B.csv 55
jump_2.0_C.csv 52
ju